# Prepare Data with Augmentation

In [ ]:
from helper_functions import load_patient, extract_training_windows, get_noise,normalize_bound
import numpy as np
import os
from tqdm import tqdm


#_______________________________________________
# Initializations
#_______________________________________________

# Window/Segment length 
l = 20 #seconds
# window stride for testing. 
s = 10 #seconds

# Base folder path
base_path = '../'

training_data_path = base_path + 'training_data_aug/'

#_______________________________________________
# Dont change These Values
#_______________________________________________
# Sapmling frequency of ecg signal is 400 hz. (CPSC2020 Data)
fs = 400
# Window/Segment length in samples. 
win_size = l*fs
# Stride for test window in samples. 
stride = s*fs
#_______________________________________________

if not os.path.exists(training_data_path):
    os.makedirs(training_data_path)



ma = np.load(base_path +'data/ma.npy')
bw = np.load(base_path +'data/bw.npy')


aug_factor = 1 # How many times to augment.



for pat_num in range(1,11):


    #_______________________________________________
    # Training Data Preparation
    #_______________________________________________
    # Load 1 patient ecg and annotations
    ecg, R_ann, S_ann, V_ann = load_patient(base_path, pat_num)
    X_train, y_train, R_w, S_w, V_w = extract_training_windows(ecg, R_ann, S_ann, V_ann, win_size)

    print('Total Windows : ', len(X_train))

    # Indexes of windows where S beats are present.
    s_w_idx = []
    for idx,ann in enumerate(S_w):    
        if ann.any():
            s_w_idx.append(idx)
    s_w_idx = np.asarray(s_w_idx, dtype=np.int32)


    # Indexes of windows where V beats are present.
    v_w_idx = []
    for idx,ann in enumerate(V_w):    
        if ann.any():
            v_w_idx.append(idx)
    v_w_idx = np.asarray(v_w_idx, dtype=np.int32)

    # Indexes of windows for V beats and S beats combined.
    sv_idx = np.unique(np.concatenate((s_w_idx,v_w_idx)))
    # All indexes of training windows.
    idx = np.arange(len(X_train))
    # indexes other than S and V beats. (Normal R peaks)
    rem_idx = np.delete(idx, sv_idx, 0)
    
    #_____________________________________________________________________
    # Choose 50% of remaining. 
    #norm_idx = np.random.choice(rem_idx, size=int(len(rem_idx)*(1/2)), replace=False)
    
    norm_idx = rem_idx[::2]
    
    # Placeholder for augmented beats
    X_aug = np.zeros((int(len(sv_idx)*aug_factor), win_size))
    y_aug = np.zeros((int(len(sv_idx)*aug_factor), win_size))
    
    count = 0
    for i in tqdm(range(len(sv_idx))):

        current_window = X_train[sv_idx[i]]
        current_window = np.squeeze(current_window)

        for j in range(aug_factor):

            noise = get_noise(ma, bw, win_size)
            aug_window = current_window + noise

            X_aug[count] = normalize_bound(aug_window, lb=-1, ub=1)

            y_aug[count] = np.squeeze(y_train[sv_idx[i]])

            count += 1
            
    X_aug = np.expand_dims(X_aug, axis=2)
    y_aug = np.expand_dims(y_aug, axis=2)
    
    
    #X_train = np.concatenate((X_train,X_aug))
    #y_train = np.concatenate((y_train,y_aug))
    
    X_train = np.concatenate((X_train[norm_idx],X_train[sv_idx], X_aug))
    y_train = np.concatenate((y_train[norm_idx],y_train[sv_idx], y_aug))


    assert len(X_train) == len(y_train)

    print('Normal Beat Windows : ', len(rem_idx))
    print('Abnormal Beat Windows : ', len(sv_idx))
    print('Augmented abnormal Beat Windows : ', len(X_aug))

    print('Saving Data')
    f_X = training_data_path+ 'X_train_P' + str(pat_num).zfill(2) + '.npy'
    f_y = training_data_path+ 'y_train_P' + str(pat_num).zfill(2) + '.npy'
    np.save(f_X, X_train)
    np.save(f_y, y_train)
    print('Done..')

# Training and Evaluation

In [2]:
from helper_functions import train_selfONN, test_selfONN

from pathlib import Path
import pandas as pd 

import torch
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

import time
import os

Q = [3]
all_patients = [1,2,3,4,5,6,7,8.9,10]
base_path = '../'
num_epochs = 30

results_all = np.zeros((10*len(Q),6), dtype = np.int32)
perc_all = np.zeros((10*len(Q),3), dtype = np.float32)
results_S = np.zeros((10*len(Q),5), dtype = np.int32)
results_V = np.zeros((10*len(Q),5), dtype = np.int32)

count = 0


for pat_num in [1]:

    for q in Q:
        
        
        # Training
        #train_selfONN(pat_num, q, epochs = num_epochs)
        
        # Testing
        stats_R, stats_S, stats_V = test_selfONN(pat_num, q, threshold = 0.1)

        #_______________________________________________
        # Saving stats
        #_______________________________________________

        if not os.path.exists(base_path + 'Results'):
            os.makedirs(base_path + 'Results')

        # Save Results for All beats
        results_all[count][0] = pat_num
        results_all[count][1] = q
        results_all[count][2:] = stats_R[:4]
        perc_all[count] = stats_R[4:7]

        df_all = pd.DataFrame(results_all)
        df_all = pd.concat([df_all, pd.DataFrame(perc_all, dtype = np.float32)], axis=1)
        df_all.columns = ['Patient No', 'Q', 'Total Beats', 'TP', 'FN', 'FP', 'Recall', 'Precision', 'F1']
        f = base_path + 'Results/selfONN_all_3a.csv'
        df_all.to_csv (r'{}'.format(f), index = False, header=True)

        
        # Save Results for S beats
        if stats_S != []:
            results_S[count][0] = pat_num
            results_S[count][1] = q
            results_S[count][2:] = stats_S[:3]

            df_S = pd.DataFrame(results_S)
            df_S.columns = ['Patient No', 'Q','Total Beats', 'Detected', 'Missed']
            f = base_path + 'Results/selfONN_S_3a.csv'
            df_S.to_csv (r'{}'.format(f), index = False, header=True)

        # Save Results for V beats
        if stats_V != []:

            results_V[count][0] = pat_num
            results_V[count][1] = q
            results_V[count][2:] = stats_V[:3]

            df_V = pd.DataFrame(results_V)
            df_V.columns = ['Patient No', 'Q','Total Beats', 'Detected', 'Missed']
            f = base_path + 'Results/selfONN_V_3a.csv'
            df_V.to_csv (r'{}'.format(f), index = False, header=True)

        count += 1 


******************************************
Testing
Patient:  1
Q :  3
******************************************
______________________________________________
Loading Data for Patient : 1
______________________________________________
Total Beats :  109731
S beats :  24
V beats :  0


100%|██████████| 110977/110977 [01:48<00:00, 1026.56it/s]


109292
______________________________________________
All Beats
______________________________________________
_________Calculating Stats____________________
______________________________________________
TP's:108908 FN's:823 FP's:397
Recall:99.25, Precision(FNR):99.64, F1-Score:99.44
Total 109731
______________________________________________
S Beats
______________________________________________
_________Calculating Stats____________________
______________________________________________
TP's:24 FN's:0 FP's:96414
Recall:100.0, Precision(FNR):0.02, F1-Score:0.04
Total 24
